In [1]:
import sys
sys.path.append('../ARC/')
sys.path.append('../RMG-Py/')

In [2]:
import torch
import torch_geometric as tg
from torch_geometric.data import DataLoader

In [3]:
import copy

from rmgpy.molecule.converter import to_rdkit_mol
from rmgpy.molecule.molecule import *
from rmgpy.species import Species
from rdkit import Chem
from rdkit.Chem.rdchem import ChiralType
from rdkit.Chem import AllChem
from typing import List, Tuple, Union

from arc.species import ARCSpecies
from arc.reaction import ARCReaction
from arc.species.converter import (check_isomorphism,
                                   molecules_from_xyz,
                                   str_to_xyz,
                                   xyz_to_str,
                                   xyz_to_x_y_z,
                                   xyz_to_xyz_file_format,
                                   xyz_to_dmat,
                                   xyz_to_coords_list,
                                  )
from IPython.display import display
import arc
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib inline

## Assume we have atom-mapped and optimized reactants and products...

In [4]:
# inputs

xyz1 = """C  -1.3087    0.0068    0.0318
C  0.1715   -0.0344    0.0210
N  0.9054   -0.9001    0.6395
O  2.1683   -0.5483    0.3437
N  2.1499    0.5449   -0.4631
N  0.9613    0.8655   -0.6660
H  -1.6558    0.9505    0.4530
H  -1.6934   -0.0680   -0.9854
H  -1.6986   -0.8169    0.6255"""

xyz2 = """C  -1.0108   -0.0114   -0.0610  
C  0.4780    0.0191    0.0139    
N  1.2974   -0.9930    0.4693    
O  0.6928   -1.9845    0.8337    
N  1.7456    1.9701   -0.6976    
N  1.1642    1.0763   -0.3716    
H  -1.4020    0.9134   -0.4821  
H  -1.3327   -0.8499   -0.6803   
H  -1.4329   -0.1554    0.9349"""

reactant = ARCSpecies(label='reactant', smiles='C(c1nonn1)([H])([H])[H]', multiplicity=1, charge=0, xyz=xyz1)
product = ARCSpecies(label='product', smiles='[N-]=[N+]=C(N=O)C', multiplicity=1, charge=0, xyz=xyz2)

rxn = ARCReaction(label='reactant <=> product')
rxn.r_species = [reactant]
rxn.p_species = [product]

reactant = rxn.r_species[0]
product = rxn.p_species[0]

print(reactant.get_xyz())
print('\n')
print(product.get_xyz())

{'symbols': ('C', 'C', 'N', 'O', 'N', 'N', 'H', 'H', 'H'), 'isotopes': (12, 12, 14, 16, 14, 14, 1, 1, 1), 'coords': ((-1.3087, 0.0068, 0.0318), (0.1715, -0.0344, 0.021), (0.9054, -0.9001, 0.6395), (2.1683, -0.5483, 0.3437), (2.1499, 0.5449, -0.4631), (0.9613, 0.8655, -0.666), (-1.6558, 0.9505, 0.453), (-1.6934, -0.068, -0.9854), (-1.6986, -0.8169, 0.6255))}


{'symbols': ('C', 'C', 'N', 'O', 'N', 'N', 'H', 'H', 'H'), 'isotopes': (12, 12, 14, 16, 14, 14, 1, 1, 1), 'coords': ((-1.0108, -0.0114, -0.061), (0.478, 0.0191, 0.0139), (1.2974, -0.993, 0.4693), (0.6928, -1.9845, 0.8337), (1.7456, 1.9701, -0.6976), (1.1642, 1.0763, -0.3716), (-1.402, 0.9134, -0.4821), (-1.3327, -0.8499, -0.6803), (-1.4329, -0.1554, 0.9349))}


In [ ]:
for a1, atom in enumerate(reactant.mol.atoms):
    print(atom.atomtype)

### generate features
- put this in `ts_gen_v2` as a function that can be imported by ARC
- **or put this as a method in the `gnn_isomerization` TS adapter in ARC**

In [9]:
from ts_gen_v2_arc.arc_featurization import featurization
tg_data = featurization(rxn)
tg_data

Data(edge_attr=[72, 2], edge_index=[2, 72], x=[9, 26])

### generate TS guess

Set paths to the optimal model weights. Note that these weights are for a model trained only on the training set
- we should retrain on the entire dataset with the optimal hyperparameters 
- we should retrain using the more advanced featurizaiton

In [ ]:
from model.G2C import G2C

# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set paths to the optimal model weights. 
# fix the typo in the name: paramAters to paramEters
yaml_file_name = os.path.join(ts_gen_v2_path, 'best_model', 'model_paramaters.yml')
state_dict = os.path.join(ts_gen_v2_path, 'best_model', 'epoch_95_state_dict')

In [ ]:
# create the network with the best architecture from hyperopt and load the corresponding best weights
with open(yaml_file_name, 'r') as f:
    content = yaml.load(stream=f, Loader=yaml.FullLoader)
print(content)

model = G2C(**content).to(device)
model.load_state_dict(torch.load(state_dict, map_location=device))
model.eval()

In [ ]:
# create data loader
data_list = list()           # list of tg.data.Data objects
ts_xyz_dict_list = list()    # list of standard ARC xyz dictionaries for the TS
for reaction in arc_reactions:
    data, ts_xyz_dict = ARC_featurization(reaction)
    data_list.append(data)
    ts_xyz_dict_list.append(ts_xyz_dict)
    
loader = DataLoader(data_list, batch_size=16)

In [ ]:
coords_list = list()
index = 0
for data in loader:
    data = data.to(device)
    out, mask = model(data)  # out is distance matrix. mask is matrix of 1s with 0s along diagonal
    
    # convert distance matrix to xyz
    ts_guess_xyz = model.low_rank_approx_power(out)
    # print(ts_guess_xyz)

    # extract xyz coordinates
    # shape of data.coords is (batch_size, n_atoms, 3). Each batch entry holds 1 TS guess
    for batch in data.coords:
        coords = batch.double().cpu().detach().numpy().tolist()
        ts_guess_coords = tuple()
        for atom in coords:  # range(coords.shape[0]):
            # unpack values and convert to immutable tuple
            x, y, z = atom
            coord = (x, y, z)
            ts_guess_coords += (coord,)
        ts_xyz_dict_list[index]['coords'] = ts_guess_coords
        index += 1
ts_xyz_dict_list

In [ ]:
# show that we can convert the ARC xyz dictionary to a standard xyz file format
print(xyz_to_xyz_file_format(ts_xyz_dict_list[-1]))

Try re-ordering atoms in case things are out of order
- Useful for when users just provide SMILES string to ARC rather than specifying the xyz coordinates. But currently, this relies on RDKit sorting the atoms properly, so it doesn't not even use the `final_xyz` attribute on all ARCSpecies objects...

In [ ]:
import sys
# define path to rmsd: https://github.com/charnley/rmsd
rmsd_path = os.path.join(os.path.dirname(arc_path), 'rmsd/')
sys.path.append(rmsd_path)

In [ ]:
import rmsd

In [ ]:
reactant_xyz = np.array(xyz_to_coords_list(reactant.get_xyz()))
reactant_xyz 

In [ ]:
ts_guess_array = np.array(xyz_to_coords_list(ts_xyz_dict_list[14]))
ts_guess_array

In [ ]:
rmsd.rmsd(reactant_xyz, ts_guess_array)